In [4]:
# 1. 환경설정 및 데이터 로드
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 데이터 로드
try:
    df = pd.read_csv('../../data/secom.csv')
    print("Data loaded successfully.")
except FileNotFoundError:
    print("File not found. Please check the file path.")

Data loaded successfully.


In [26]:
# 2. 데이터  정보 확인 (project_defect 에서 확인한 정보 외 추가확인)

# 데이터 크기
print(f'행(샘플) 수: {df.shape[0]}')
print(f'열(특성) 수: {df.shape[1]}')

# Time 타입 확인
print(df['Time'].dtype)
print(df['Time'].head())

# 기간 및 파생변수 확인
df['Time'] = pd.to_datetime(df['Time'])
print(f"데이터 기간: {df['Time'].min()} ~ {df['Time'].max()}")

# datetime 변환
df['Time'] = pd.to_datetime(df['Time'])
df['Hour'] = df['Time'].dt.hour
df['DayOfWeek'] = df['Time'].dt.dayofweek
df['YearMonth'] = df['Time'].dt.to_period('M')

# 시간대별 분포 (24시간 전체)
print("\n시간대별 분포 (0-23시 전체):")
print("-" * 40)
hourly_pct = (df['Hour'].value_counts().sort_index() / len(df) * 100).round(2)
for hour, pct in hourly_pct.items():
    print(f"{hour:2d}시: {pct:5.2f}%")

# 요일별 분포 (월-일 전체)
print("\n요일별 분포 (월-일 전체):")
print("-" * 40)
dow_labels = ['월', '화', '수', '목', '금', '토', '일']
dow_pct = (df['DayOfWeek'].value_counts().sort_index() / len(df) * 100).round(2)
for dow, pct in dow_pct.items():
    print(f"{dow_labels[dow]:2s}: {pct:5.2f}%")

# 월별 분포 (전체)
print("\n월별 분포 (전체):")
print("-" * 40)
monthly_pct = (df['YearMonth'].value_counts().sort_index() / len(df) * 100).round(2)
for month, pct in monthly_pct.items():
    print(f"{month}: {pct:5.2f}%")

# 결론: 시간대는 비교적 고르게 분포, 요일은 약간의 차이, 월별은 특정월(8,9) 집중
# 불량률 데이터 분석시 시간대, 요일, 월별 편차 고려 필요

행(샘플) 수: 1567
열(특성) 수: 600
datetime64[ns]
0   2008-07-19 11:55:00
1   2008-07-19 12:32:00
2   2008-07-19 13:17:00
3   2008-07-19 14:43:00
4   2008-07-19 15:22:00
Name: Time, dtype: datetime64[ns]
데이터 기간: 2008-01-08 02:02:00 ~ 2008-12-10 18:47:00

시간대별 분포 (0-23시 전체):
----------------------------------------
 0시:  4.15%
 1시:  2.68%
 2시:  3.83%
 3시:  4.47%
 4시:  4.15%
 5시:  4.72%
 6시:  3.96%
 7시:  4.66%
 8시:  4.08%
 9시:  3.51%
10시:  3.38%
11시:  4.34%
12시:  4.28%
13시:  3.19%
14시:  4.91%
15시:  6.32%
16시:  3.70%
17시:  3.70%
18시:  4.02%
19시:  4.02%
20시:  5.04%
21시:  4.53%
22시:  3.57%
23시:  4.79%

요일별 분포 (월-일 전체):
----------------------------------------
월 : 11.68%
화 : 14.93%
수 : 11.93%
목 : 16.02%
금 : 12.00%
토 : 18.32%
일 : 15.12%

월별 분포 (전체):
----------------------------------------
2008-01:  3.25%
2008-02:  6.25%
2008-03:  3.19%
2008-04:  3.13%
2008-05:  3.96%
2008-06:  4.28%
2008-07:  7.28%
2008-08: 30.06%
2008-09: 26.36%
2008-10:  7.85%
2008-11:  2.23%
2008-12:  2.17%


In [ ]:
# 3. 불량률 데이터 분석

target_col = 'Pass/Fail'  # project_defect에서 확인한 정보

# 시간대별 불량률 계산
hourly_defect_rate = df.groupby('Hour')[target_col].apply(
    lambda x: (x == 1).sum() / len(x) * 100
).round(2)
print("\n시간대별 불량률 (%):")
print(hourly_defect_rate)

# 요일별 불량률
dow_defect_rate = df.groupby('DayOfWeek')[target_col].apply(
    lambda x: (x == 1).sum() / len(x) * 100
).round(2)

dow_labels = ['월', '화', '수', '목', '금', '토', '일']
print("\n요일별 불량률 (%):")
for dow, rate in dow_defect_rate.items():
    print(f"{dow_labels[dow]}: {rate}%")

# 월별 불량률
monthly_defect_rate = df.groupby('YearMonth')[target_col].apply(
    lambda x: (x == 1).sum() / len(x) * 100
).round(2)

print("\n월별 불량률 (%):")
print(monthly_defect_rate)

# 결론: 불량률 시간/요일/월별 차이 큼 (최대 7배), 샘플 적은 구간(1시, 12월) 신뢰도 낮음
# -> 샘플 많은 위험 시간대(15시 6.32%, 수요일 11.93%, 7월 7.28%) 중심 센서 분석, 동적 기준선 필요



시간대별 불량률 (%):
Hour
0      4.62
1      7.14
2      5.00
3      7.14
4      9.23
5      5.41
6      8.06
7      4.11
8      4.69
9     10.91
10     1.89
11     7.35
12     4.48
13    10.00
14     3.90
15    11.11
16     6.90
17     3.45
18     4.76
19     6.35
20     7.59
21     9.86
22     7.14
23     6.67
Name: Pass/Fail, dtype: float64

요일별 불량률 (%):
월: 6.01%
화: 7.69%
수: 10.16%
목: 3.59%
금: 6.38%
토: 5.57%
일: 8.02%

월별 불량률 (%):
YearMonth
2008-01     5.88
2008-02     5.10
2008-03     2.00
2008-04     6.12
2008-05    11.29
2008-06     8.96
2008-07    14.04
2008-08     8.07
2008-09     4.12
2008-10     4.88
2008-11     5.71
2008-12     0.00
Freq: M, Name: Pass/Fail, dtype: float64


# 데이터 탐색 결과 요약

## 데이터 기본 정보
- **기간**: 2008년 1월 ~ 12월 (약 11개월)
- **샘플 수**: 1,567개
- **센서 수**: 590개
- **타겟**: Pass/Fail (정상 93.4%, 불량 6.6%)

---

## OEE 구성요소별 발견

### 1. 가동률 (Availability)
- **운영 방식**: 24시간/주7일 풀가동
- **시간대별 분포**: 비교적 균등 (2.68~6.32%)
- **가동 중단**: 데이터 기록 중단 없음
- **예상 가동률**: 85% 이상 (양호)

### 2. 양품률 (Quality)
- **전체 양품률**: 93.4% (불량 6.6%)
- **시간대별 차이**: 
  - 최고: 10시 98.1% (불량 1.89%)
  - 최저: 15시 88.9% (불량 11.11%)
  - 차이: 약 9.2%p
- **요일별 차이**:
  - 최고: 목요일 96.4% (불량 3.59%)
  - 최저: 수요일 89.8% (불량 10.16%)
  - 차이: 약 6.6%p
- **월별 차이**:
  - 최고: 12월 100% (불량 0%, 단 샘플 34개)
  - 최저: 7월 86.0% (불량 14.04%)
  - 차이: 약 14%p

### 3. 성능률 (Performance)
- **생산 속도 데이터**: 없음 (SECOM 데이터 특성)
- **처리 방식**: 업계 평균 80% 가정 또는 제외

---

## 주요 문제점

### 문제 1: 양품률 시간대별 편차 큼
- **15시**: 88.9% (최저)
- **10시**: 98.1% (최고)
- **영향**: 시간대별 OEE 최대 7.8%p 차이 예상
- **심각도**: 높음 (개선 여지 큼)

### 문제 2: 요일 효과
- **수요일**: 89.8% (주중 최저)
- **목요일**: 96.4% (주중 최고)
- **영향**: 요일별 생산 계획 필요
- **심각도**: 중간

### 문제 3: 계절 효과
- **7월**: 86.0% (하절기 품질 저하)
- **3월**: 98.0% (양호)
- **영향**: 하절기 집중 관리 필요
- **심각도**: 높음

### 문제 4: 데이터 신뢰도
- **12월**: 샘플 34개 (신뢰도 낮음)
- **8-9월**: 전체의 56% (편중)
- **영향**: 연간 OEE 추정 시 주의
- **심각도**: 중간

---

## 예상 OEE

### 기준 계산 (전체 평균)
OEE= 가동률x성능률x양품률
= 0.85 x 0.80 x 0.934
= 63.5%

### 시간대별 예상 OEE
- **10시**: 0.85 × 0.80 × 0.981 = 66.8% (최고)
- **15시**: 0.85 × 0.80 × 0.889 = 60.5% (최저)
- **차이**: 6.3%p

### 요일별 예상 OEE
- **목요일**: 0.85 × 0.80 × 0.964 = 65.6%
- **수요일**: 0.85 × 0.80 × 0.898 = 61.1%
- **차이**: 4.5%p

---

## 다음 단계 계획

### 02_oee_calculation.ipynb
1. **가동률 정밀 계산**
   - 시간대별 가동 패턴 분석
   - 정지 시간 추정

2. **성능률 추정**
   - 업계 표준 적용 (80%)
   - 또는 생산량 데이터 있으면 계산

3. **OEE 계산 및 시각화**
   - 시간대/요일/월별 OEE
   - 손실 분석 (어디서 효율 떨어지나?)

4. **개선 방안 제시**
   - 양품률 개선 포인트 (15시/수/7월)
   - 프로젝트 2 예측 모델 활용 방안
   - 목표 OEE 설정 (70%+)

---

## 결론

**주요 발견**: 양품률이 시간대/요일/월별로 큰 차이 (최대 14%p)
**OEE 예상**: 약 63.5% (업계 평균 65-75% 대비 낮음)
**개선 포인트**: 15시/수요일/7월 양품률 개선 → OEE 5-10%p 향상 가능
**활용**: 불량 예측 모델과 연계한 품질 개선 전략 수립